In [1]:
using Revise

using Langevin
using Langevin.Preconditioners: MatrixMOp, BlockPreconditioner, FourierPreconditioner

using LinearAlgebra
using IterativeSolvers
using SparseArrays
using Random

import Serialization
holstein = Serialization.deserialize("holstein_6x6_cond106.dat")

M = Array(Langevin.HolsteinModels.construct_M(holstein))
M_op = MatrixMOp(holstein)
P_op = BlockPreconditioner(holstein, subtol=1e-2)
PF_op = FourierPreconditioner(holstein)
;

In [ ]:
cond(M)

# Accuracy measurements

In [18]:
Random.seed!(0)
b = randn(size(holstein)[1])

x_exact = M \ b
x_approx = fill!(similar(b), 0)
;

In [25]:
rhs = M' * b
fill!(x_approx, 0)
x1 = IterativeSolvers.cg!(x_approx, holstein, rhs, tol=1e-4, log=true, maxiter=1000)
println("CG iters ", x1[2].iters)
println("True error ", norm(x_approx - x_exact) / norm(x_exact))

CG iters 286
True error 0.005060902377033015


In [26]:
fill!(x_approx, 0)
x2 = IterativeSolvers.gmres!(x_approx, M_op, b, Pr=P_op, tol=2e-3, restart=10, log=true, maxiter=100, initially_zero=true, verbose=true)

println("GMRES with block preconditioner")
println("True error ", norm(x_approx - x_exact) / norm(x_exact))

=== gmres ===
rest	iter	resnorm
  1	  1	8.34e+00
  1	  2	2.88e+00
  1	  3	1.29e+00
  1	  4	4.81e-01
  1	  5	2.05e-01
  1	  6	7.68e-02

GMRES with block preconditioner
True error 0.004140905302401464


In [27]:
fill!(x_approx, 0)
x2 = IterativeSolvers.gmres!(x_approx, M_op, b, Pr=PF_op, tol=2e-3, restart=10, log=true, maxiter=100, initially_zero=true, verbose=true)

println("GMRES with Fourier preconditioner")
println("True error ", norm(x_approx - x_exact) / norm(x_exact))

=== gmres ===
rest	iter	resnorm
  1	  1	9.68e+00
  1	  2	4.28e+00
  1	  3	2.19e+00
  1	  4	1.24e+00
  1	  5	7.24e-01
  1	  6	4.14e-01
  1	  7	2.14e-01
  1	  8	9.92e-02

GMRES with Fourier preconditioner
True error 0.005586265309813534


In [52]:
fill!(x_approx, 0)
x2 = IterativeSolvers.gmres!(x_approx, M_op, b, Pl=PF_op, tol=5e-3, restart=10, log=true, maxiter=100, initially_zero=true, verbose=true)

println("GMRES with LEFT Fourier preconditioner")
println("True error ", norm(x_approx - x_exact) / norm(x_exact))

=== gmres ===
rest	iter	resnorm
  1	  1	5.06e+01
  1	  2	2.57e+01
  1	  3	1.43e+01
  1	  4	7.83e+00
  1	  5	4.62e+00
  1	  6	2.47e+00
  1	  7	1.13e+00
  1	  8	5.39e-01

GMRES with LEFT Fourier preconditioner
True error 0.0052154866371389936


In [54]:
# BiCGStab uses internal randomness. :-(
# Set seed to get reproducible results.
Random.seed!(0)

fill!(x_approx, 0)
x2 = IterativeSolvers.bicgstabl!(x_approx, M_op, b, 1, Pl=PF_op, tol=1e-2, log=true, max_mv_products=100, initial_zero=true, verbose=true)

println("BiCGStab(1) with Fourier preconditioner")
println("True error ", norm(x_approx - x_exact) / norm(x_exact))

  1	6.59e+01
  2	8.09e+00
  3	6.59e+01
  4	3.68e+00
  5	3.19e-01

BiCGStab(1) with Fourier preconditioner
True error 0.0033509204835341735


# Benchmarking

In [46]:
using BenchmarkTools

println("CG time...")
@btime begin
    fill!(x_approx, 0)
    IterativeSolvers.cg!(x_approx, holstein, b, tol=1e-4, log=true, maxiter=1000)
end
;

CG time...
  3.879 ms (1243 allocations: 95.75 KiB)


In [30]:
println("GMRES/Block time...")
@btime begin
    fill!(x_approx, 0)
    IterativeSolvers.gmres!(x_approx, M_op, b, Pr=P_op, tol=2e-3, restart=10, maxiter=100, initially_zero=true)
end
;

GMRES/Block time...
  2.032 ms (18407 allocations: 1.37 MiB)


In [31]:
println("GMRES/Fourier time...")
@btime begin
    fill!(x_approx, 0)
    IterativeSolvers.gmres!(x_approx, M_op, b, Pr=PF_op, tol=2e-3, restart=10, maxiter=100, initially_zero=true)
end
;

GMRES/Fourier time...
  658.695 μs (118 allocations: 277.47 KiB)


In [50]:
println("GMRES/Fourier time, prealloc...")

gmres = IterativeSolvers.gmres_iterable!(x_approx, M_op, b, Pr=PF_op, tol=2e-3, restart=10, maxiter=100, initially_zero=true)

@btime begin
    fill!(x_approx, 0)
    Langevin.Preconditioners.reset_gmres_iterable!(gmres, x_approx, M_op, b, tol=2e-3, initially_zero=true)
    Langevin.Preconditioners.run_gmres_iterable!(gmres)
end
;

GMRES/Fourier time, prealloc...
  545.088 μs (101 allocations: 5.20 KiB)


In [55]:
println("BiCGStab(1)/Fourier time...")

Random.seed!(0)

@btime begin
    fill!(x_approx, 0)
    x2 = IterativeSolvers.bicgstabl!(x_approx, M_op, b, 1, Pl=PF_op, tol=1e-2, max_mv_products=100, initial_zero=true)
end
;

BiCGStab(1)/Fourier time...
  503.554 μs (124 allocations: 478.97 KiB)


## For reference, cost of raw FFT and raw precond ops

In [42]:
using BenchmarkTools

L = holstein.Lτ
(N1, N2, N3) = holstein.lattice.dims
N = N1*N2*N3

s1 = randn(ComplexF64, (L, N1, N2, N3))
s2 = zeros(ComplexF64, (L, N1, N2, N3))

println("Timing FFTs")
@btime for i=1:22
    mul!(s2, PF_op.plan, s1)
end
;

Timing FFTs
  181.615 μs (0 allocations: 0 bytes)


In [41]:
r1 = randn(L*N)
r2 = zeros(L*N)

println("Timing preconditioner op")
@btime for i=1:11
    ldiv!(r2, PF_op, r1)
end

Timing preconditioner op
  399.292 μs (0 allocations: 0 bytes)


In [43]:
println("Timing matmul")
@btime for i=1:11
    mul!(r2, M_op, r1)
end

Timing matmul
  34.745 μs (0 allocations: 0 bytes)


# Profiling

For some reason, it appears that the resulting flamegraphs are not entirely trustworthy.

In [6]:
using Profile
Profile.clear()  # in case we have any previous profiling data

@profile begin
    for i = 1:100
        fill!(x_approx, 0)
        IterativeSolvers.bicgstabl!(x_approx, M_op, b, 1, Pl=PF_op, tol=1e-2, max_mv_products=100, initial_zero=true)
    end
end

In [ ]:
Serialization.serialize("serialized_profile.dat", Profile.retrieve())

In [19]:
Profile.print()

In [ ]:
r = Serialization.deserialize("serialized_profile.dat")
using ProfileView
ProfileView.view(r[1], lidict=r[2])
ProfileView.svgwrite("flamegraph_gmres.svg",r[1],r[2])